# 读取数据，查看数据，处理数据

In [1]:
import pandas as pd 
df = pd.read_csv('../data/menu.csv',encoding='utf8')
df.head()

,name,foods,score,kind
0,超级下饭的地三鲜,长茄子、青椒、土豆、蚝油、生抽、盐、糖、老抽、淀粉、葱、蒜,8.0,家常菜
1,农家一碗香（下饭菜）,青红椒、鸡蛋、五花肉、生抽、老抽、蚝油、葱姜蒜、料酒,8.1,家常菜
2,土豆焖牛腩,牛腩、土豆、胡萝卜、葱、姜、蒜、八角、香叶、桂皮、陈皮、花椒、水、腐乳（王致和大块腐乳）、腐...,9.3,家常菜
3,蚂蚁上树(肉末炒粉丝)超下饭‼️十分钟搞定,红薯粉丝、肉末(猪肉糜)、葱花、姜片、盐、料酒、酱油、豆瓣酱、糖、油,8.3,家常菜
4,和饭店一个味道的干锅花菜,花菜、肥五花、青红线椒、姜，大蒜、芹菜、料酒、盐、耗油、生抽,7.8,家常菜


In [3]:
# 查看缺失值
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5191 entries, 0 to 5190
Data columns (total 4 columns):
name     5191 non-null object
foods    5190 non-null object
score    5191 non-null float64
kind     5191 non-null object
dtypes: float64(1), object(3)
memory usage: 162.3+ KB


In [9]:
# 删除缺失值
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5190 entries, 0 to 5190
Data columns (total 4 columns):
name     5190 non-null object
foods    5190 non-null object
score    5190 non-null float64
kind     5190 non-null object
dtypes: float64(1), object(3)
memory usage: 202.7+ KB


# Apriori 算法

In [40]:
# -*- coding: utf-8 -*-

# 代码8-6 构建关联规则模型

from numpy import *


def loadDataSet():
    return [['a', 'c', 'e'], ['b', 'd'], ['b', 'c'], ['a', 'b', 'c', 'd'], ['a', 'b'], ['b', 'c'], ['a', 'b'],
            ['a', 'b', 'c', 'e'], ['a', 'b', 'c'], ['a', 'c', 'e']]


def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # 映射为frozenset唯一性的，可使用其构造字典
    return list(map(frozenset, C1))


# 从候选K项集到频繁K项集（支持度计算）
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:  # 遍历数据集
        for can in Ck:  # 遍历候选项
            if can.issubset(tid):  # 判断候选项中是否含数据集的各项
                if not can in ssCnt:
                    ssCnt[can] = 1  # 不含设为1
                else:
                    ssCnt[can] += 1  # 有则计数加1
    numItems = float(len(D))  # 数据集大小
    retList = []  # L1初始化
    supportData = {}  # 记录候选项中各个数据的支持度
    for key in ssCnt:
        support = ssCnt[key] / numItems  # 计算支持度
        if support >= minSupport:
            retList.insert(0, key)  # 满足条件加入L1中
            supportData[key] = support
    return retList, supportData


def calSupport(D, Ck, min_support):
    dict_sup = {}
    for i in D:
        for j in Ck:
            if j.issubset(i):
                if not j in dict_sup:
                    dict_sup[j] = 1
                else:
                    dict_sup[j] += 1
    sumCount = float(len(D))
    supportData = {}
    relist = []
    for i in dict_sup:
        temp_sup = dict_sup[i] / sumCount
        if temp_sup >= min_support:
            relist.append(i)
            # 此处可设置返回全部的支持度数据（或者频繁项集的支持度数据）
            supportData[i] = temp_sup
    return relist, supportData


# 改进剪枝算法
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):  # 两两组合遍历
            L1 = list(Lk[i])[:k - 2]
            L2 = list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:  # 前k-1项相等，则可相乘，这样可防止重复项出现
                # 进行剪枝（a1为k项集中的一个元素，b为它的所有k-1项子集）
                a = Lk[i] | Lk[j]  # a为frozenset()集合
                a1 = list(a)
                b = []
                # 遍历取出每一个元素，转换为set，依次从a1中剔除该元素，并加入到b中
                for q in range(len(a1)):
                    t = [a1[q]]
                    tt = frozenset(set(a1) - set(t))
                    b.append(tt)
                t = 0
                for w in b:
                    # 当b（即所有k-1项子集）都是Lk（频繁的）的子集，则保留，否则删除。
                    if w in Lk:
                        t += 1
                if t == len(b):
                    retList.append(b[0] | b[1])
    return retList


def apriori(dataSet, minSupport=0.2):
    # 前3条语句是对计算查找单个元素中的频繁项集
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))  # 使用list()转换为列表
    L1, supportData = calSupport(D, C1, minSupport)
    L = [L1]  # 加列表框，使得1项集为一个单独元素
    k = 2
    while (len(L[k - 2]) > 0):  # 是否还有候选集
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minSupport)  # scan DB to get Lk
        supportData.update(supK)  # 把supk的键值对添加到supportData里
        L.append(Lk)  # L最后一个值为空集
        k += 1
    del L[-1]  # 删除最后一个空集
    return L, supportData  # L为频繁项集，为一个列表，1，2，3项集分别为一个元素


# 生成集合的所有子集
def getSubset(fromList, toList):
    for i in range(len(fromList)):
        t = [fromList[i]]
        tt = frozenset(set(fromList) - set(t))
        if not tt in toList:
            toList.append(tt)
            tt = list(tt)
            if len(tt) > 1:
                getSubset(tt, toList)


def calcConf(freqSet, H, supportData, ruleList, minConf=0.7):
    for conseq in H:  # 遍历H中的所有项集并计算它们的可信度值
        conf = supportData[freqSet] / supportData[freqSet - conseq]  # 可信度计算，结合支持度数据
        # 提升度lift计算lift = p(a & b) / p(a)*p(b)
        lift = supportData[freqSet] / (supportData[conseq] * supportData[freqSet - conseq])

        if conf >= minConf and lift > 1:
            print(freqSet - conseq, '-->', conseq, '支持度', round(supportData[freqSet], 6), '置信度：', round(conf, 6),
                  'lift值为：', round(lift, 6))
            ruleList.append((freqSet - conseq, conseq, conf))


# 生成规则
def gen_rule(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):  # 从二项集开始计算
        for freqSet in L[i]:  # freqSet为所有的k项集
            # 求该三项集的所有非空子集，1项集，2项集，直到k-1项集，用H1表示，为list类型,里面为frozenset类型，
            H1 = list(freqSet)
            all_subset = []
            getSubset(H1, all_subset)  # 生成所有的子集
            calcConf(freqSet, all_subset, supportData, bigRuleList, minConf)
    return bigRuleList

# 调用算法

In [41]:
dataset = []
data = list(df['foods'])
for d in data:
    d = d.split('、')
    dataset.append(d)

L, supportData = apriori(dataset, minSupport=0.02) # 最小支持度
rule = gen_rule(L, supportData, minConf=0.35) # 最小置信度

# 输出支持度，置信度大于最小值的规则
rule

frozenset({'细砂糖'}) --> frozenset({'低筋面粉'}) 支持度 0.027746 置信度： 0.377953 lift值为： 5.872978
frozenset({'低筋面粉'}) --> frozenset({'细砂糖'}) 支持度 0.027746 置信度： 0.431138 lift值为： 5.872978
frozenset({'酵母'}) --> frozenset({'牛奶'}) 支持度 0.025626 置信度： 0.378917 lift值为： 3.621696
frozenset({'高筋面粉'}) --> frozenset({'牛奶'}) 支持度 0.020617 置信度： 0.551546 lift值为： 5.271687
frozenset({'奶粉'}) --> frozenset({'盐'}) 支持度 0.020424 置信度： 0.773723 lift值为： 1.548639
frozenset({'高筋面粉'}) --> frozenset({'黄油'}) 支持度 0.023699 置信度： 0.634021 lift值为： 7.670319
frozenset({'细砂糖'}) --> frozenset({'鸡蛋'}) 支持度 0.035067 置信度： 0.47769 lift值为： 1.705098
frozenset({'低筋面粉'}) --> frozenset({'牛奶'}) 支持度 0.028709 置信度： 0.446108 lift值为： 4.263903
frozenset({'低筋面粉'}) --> frozenset({'鸡蛋'}) 支持度 0.037958 置信度： 0.58982 lift值为： 2.105342
frozenset({'细砂糖'}) --> frozenset({'牛奶'}) 支持度 0.037187 置信度： 0.506562 lift值为： 4.841722
frozenset({'牛奶'}) --> frozenset({'细砂糖'}) 支持度 0.037187 置信度： 0.355433 lift值为： 4.841722
frozenset({'黄油'}) --> frozenset({'牛奶'}) 支持度 0.04027 置信度： 0.487

frozenset({'盐', '葱'}) --> frozenset({'料酒'}) 支持度 0.034489 置信度： 0.438725 lift值为： 2.456295
frozenset({'葱', '料酒'}) --> frozenset({'盐'}) 支持度 0.034489 置信度： 0.716 lift值为： 1.433105
frozenset({'料酒', '蒜'}) --> frozenset({'盐'}) 支持度 0.028324 置信度： 0.703349 lift值为： 1.407784
frozenset({'蚝油', '料酒'}) --> frozenset({'盐'}) 支持度 0.026012 置信度： 0.61086 lift值为： 1.222662
frozenset({'盐', '姜'}) --> frozenset({'生抽'}) 支持度 0.046435 置信度： 0.466151 lift值为： 1.776302
frozenset({'生抽', '姜'}) --> frozenset({'盐'}) 支持度 0.046435 置信度： 0.688571 lift值为： 1.378205
frozenset({'糖', '姜'}) --> frozenset({'盐'}) 支持度 0.023892 置信度： 0.765432 lift值为： 1.532045
frozenset({'盐', '葱'}) --> frozenset({'姜'}) 支持度 0.046243 置信度： 0.588235 lift值为： 3.929139
frozenset({'盐', '姜'}) --> frozenset({'葱'}) 支持度 0.046243 置信度： 0.464217 lift值为： 3.885942
frozenset({'葱'}) --> frozenset({'盐', '姜'}) 支持度 0.046243 置信度： 0.387097 lift值为： 3.885942
frozenset({'葱', '姜'}) --> frozenset({'盐'}) 支持度 0.046243 置信度： 0.701754 lift值为： 1.404591
frozenset({'盐', '蒜'}) --> frozenset({'姜'

[(frozenset({'细砂糖'}), frozenset({'低筋面粉'}), 0.3779527559055118),
 (frozenset({'低筋面粉'}), frozenset({'细砂糖'}), 0.4311377245508982),
 (frozenset({'酵母'}), frozenset({'牛奶'}), 0.3789173789173789),
 (frozenset({'高筋面粉'}), frozenset({'牛奶'}), 0.5515463917525774),
 (frozenset({'奶粉'}), frozenset({'盐'}), 0.7737226277372263),
 (frozenset({'高筋面粉'}), frozenset({'黄油'}), 0.634020618556701),
 (frozenset({'细砂糖'}), frozenset({'鸡蛋'}), 0.4776902887139107),
 (frozenset({'低筋面粉'}), frozenset({'牛奶'}), 0.4461077844311377),
 (frozenset({'低筋面粉'}), frozenset({'鸡蛋'}), 0.5898203592814371),
 (frozenset({'细砂糖'}), frozenset({'牛奶'}), 0.5065616797900262),
 (frozenset({'牛奶'}), frozenset({'细砂糖'}), 0.35543278084714547),
 (frozenset({'黄油'}), frozenset({'牛奶'}), 0.4871794871794872),
 (frozenset({'牛奶'}), frozenset({'黄油'}), 0.3848987108655617),
 (frozenset({'玉米油'}), frozenset({'鸡蛋'}), 0.5495049504950495),
 (frozenset({'黄油'}), frozenset({'鸡蛋'}), 0.46386946386946387),
 (frozenset({'黄油'}), frozenset({'盐'}), 0.634032634032634),
 (frozen

# 结果说明

随机抽取一条结果解读

frozenset({'细砂糖', '鸡蛋'}) --> frozenset({'牛奶'}) 支持度 0.021773 置信度： 0.620879

表示食材中同时有细砂糖/鸡蛋，牛奶的概率为62%，而这种按情况发生的概率为2.1%